<a href="https://colab.research.google.com/github/Maxxx-VS/CV-Arduino/blob/main/cat_for_robot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
# Загрузка данных CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Нормализация данных (приведение к диапазону [0, 1])
X_train = X_train / 255.0
X_test = X_test / 255.0

# Преобразование меток в one-hot encoding
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [13]:
# Создание модели
model = Sequential([
    Conv2D(128, (3, 3), activation='relu', input_shape=(32, 32, 3)),  # Первый сверточный слой
    MaxPooling2D((2, 2)),  # Слой подвыборки

    Conv2D(256, (3, 3), activation='relu'),  # Второй сверточный слой
    MaxPooling2D((2, 2)),  # Слой подвыборки

    Conv2D(1024, (3, 3), activation='relu'),  # Третий сверточный слой
    MaxPooling2D((2, 2)),  # Слой подвыборки

    Conv2D(2048, (3, 3), activation='relu'),  # Третий сверточный слой
    MaxPooling2D((2, 2)),  # Слой подвыборки

    Flatten(),  # Преобразование в одномерный вектор
    Dense(2048, activation='relu'),  # Полносвязный слой

    Dropout(0.5),  # Dropout для предотвращения переобучения
    Dense(10, activation='softmax')])  # Выходной слой для 10 классов

In [14]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)                    │ (None, 30, 30, 128)         │           3,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 15, 15, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_4 (Conv2D)                    │ (None, 13, 13, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_4 (MaxPooling2D)       │ (None, 6, 6, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 4, 4, 1024)          │       2,360,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_5 (MaxPooling2D)       │ (None, 2, 2, 1024)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 0, 0, 2048)          │      18,876,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 0, 0, 2048)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 0)                   │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 2048)                │           2,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 10)                  │          20,490 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 21,558,026 (82.24 MB)

 Trainable params: 21,558,026 (82.24 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
# Компиляция модели
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
# Ранняя остановка (если точность на валидации не улучшается)
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [17]:
# Обучение модели
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping])

Epoch 1/100


ValueError: Exception encountered when calling Conv2D.call().

[1mNegative dimension size caused by subtracting 3 from 2 for '{{node sequential_1_1/conv2d_6_1/convolution}} = Conv2D[T=DT_FLOAT, data_format="NHWC", dilations=[1, 1, 1, 1], explicit_paddings=[], padding="VALID", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true](sequential_1_1/max_pooling2d_5_1/MaxPool2d, sequential_1_1/conv2d_6_1/convolution/ReadVariableOp)' with input shapes: [?,2,2,1024], [3,3,1024,2048].[0m

Arguments received by Conv2D.call():
  • inputs=tf.Tensor(shape=(None, 2, 2, 1024), dtype=float32)

In [ ]:
# Оценка модели на тестовых данных
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
# Сохранение модели
model.save('cifar10_model.keras')
print("Модель сохранена как 'cifar10_model.keras'")

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np

# Загрузка модели
model = load_model('cifar10_model.keras')

# Пример предсказания
sample_image = X_test[0]  # Берем первое изображение из тестового набора
sample_image = np.expand_dims(sample_image, axis=0)  # Добавляем размерность батча
prediction = model.predict(sample_image)

# Вывод результата
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
predicted_class = class_names[np.argmax(prediction)]
print(f"Предсказанный класс: {predicted_class}")